
# Superflex Auction Draft Optimizer — 12 teams, Half-PPR, $200 (2025)

**Goal:** Maximize *expected weekly starting lineup points* under a $200 auction budget for a 12-team **half-PPR** league with **Superflex**.  
You can tune market inflation, positional spend, minimum QBs, bench weight, and a risk penalty.  
Data defaults to FantasyPros consensus (2025) with Yahoo ADP for a realism layer.

**League setup (from user):**
- Teams: 12
- Budget: $200
- Roster: QB, RB, RB, WR, WR, TE, FLEX (W/R/T), **SUPERFLEX (Q/W/R/T)**, K, DST, 6 BN
- Scoring:
  - Passing: 1 per 25 yards, TD=4, INT=-1
  - Rushing/Receiving: 1 per 10 yards, 0.5 PPR, TD=6
  - TE premium: None
  - K/DST: Yahoo standard

> **How to use**  
> 1) **Run the Setup** cell (installs deps if needed).  
> 2) **Load Data**: Either paste CSV exports (FantasyPros projections/auction values, Yahoo ADP) into the `data/` folder or point to URLs and run the fetchers.  
> 3) Tune parameters in **Config**.  
> 4) Run **Build Market** → **Optimize** → **Scenarios**.  
> 5) Use the **Outputs**: bid caps, optimal target roster, fallback ladders, value-per-dollar charts.

**Notes**  
- This notebook includes both **local CSV loaders** and **optional web fetchers**. If your runtime has no internet, use the CSV templates saved alongside this notebook.  
- The **probability-to-get** heuristic maps Yahoo ADP rank → implied price curve. You can adjust its sensitivity.


In [2]:

# === Setup ===

import os, math, json, itertools, warnings, textwrap, sys
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
import pandas as pd
import numpy as np

try:
    import pulp
except Exception as e:
    print("PuLP not found. Install with: pip install pulp")
    raise

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

os.makedirs("data", exist_ok=True)
os.makedirs("outputs", exist_ok=True)


In [3]:

# === Create CSV templates if not present ===
import pandas as pd
tpl_players = pd.DataFrame({
    "player": [],
    "team": [],
    "pos": [],
    "bye": [],
    "proj_pts": [],
    "auction_value": [],
    "yahoo_adp_rank": [],
    "risk_score": []
})
tpl_players.to_csv("data/template_players.csv", index=False)

tpl_kdst = pd.DataFrame({
    "name": [],
    "pos": [],
    "proj_pts": [],
    "auction_value": [],
    "yahoo_adp_rank": [],
    "risk_score": []
})
tpl_kdst.to_csv("data/template_kdst.csv", index=False)

print("Wrote templates to data/: template_players.csv, template_kdst.csv")


Wrote templates to data/: template_players.csv, template_kdst.csv


In [4]:

# === Config (edit these) ===
CONFIG = {
    "teams": 12,
    "budget": 200,
    "scoring": {
        "pass_yds_per_pt": 25,
        "pass_td": 4,
        "int": -1,
        "rush_yds_per_pt": 10,
        "rec_yds_per_pt": 10,
        "reception": 0.5,
        "rush_td": 6,
        "rec_td": 6,
        "te_premium_per_rec": 0.0
    },
    "starting_slots": [
        {"name": "QB", "types": ["QB"]},
        {"name": "RB1", "types": ["RB"]},
        {"name": "RB2", "types": ["RB"]},
        {"name": "WR1", "types": ["WR"]},
        {"name": "WR2", "types": ["WR"]},
        {"name": "TE", "types": ["TE"]},
        {"name": "FLEX", "types": ["RB", "WR", "TE"]},
        {"name": "SUPERFLEX", "types": ["QB", "RB", "WR", "TE"]},
        {"name": "K", "types": ["K"]},
        {"name": "DST", "types": ["DST"]},
    ],
    "bench_spots": 6,
    "min_qbs": 2,
    "bench_weight": 0.25,
    "risk_weight": 0.0,
    "global_inflation": 1.00,
    "pos_inflation": {"QB": 1.10, "RB": 1.00, "WR": 1.00, "TE": 1.00, "K": 1.00, "DST": 1.00},
    "use_adp_probability": True,
    "adp_sigmoid_width": 0.15,
    "adp_aggression": 0.0,
    "lock_positions": {},
}
print(json.dumps(CONFIG, indent=2))


{
  "teams": 12,
  "budget": 200,
  "scoring": {
    "pass_yds_per_pt": 25,
    "pass_td": 4,
    "int": -1,
    "rush_yds_per_pt": 10,
    "rec_yds_per_pt": 10,
    "reception": 0.5,
    "rush_td": 6,
    "rec_td": 6,
    "te_premium_per_rec": 0.0
  },
  "starting_slots": [
    {
      "name": "QB",
      "types": [
        "QB"
      ]
    },
    {
      "name": "RB1",
      "types": [
        "RB"
      ]
    },
    {
      "name": "RB2",
      "types": [
        "RB"
      ]
    },
    {
      "name": "WR1",
      "types": [
        "WR"
      ]
    },
    {
      "name": "WR2",
      "types": [
        "WR"
      ]
    },
    {
      "name": "TE",
      "types": [
        "TE"
      ]
    },
    {
      "name": "FLEX",
      "types": [
        "RB",
        "WR",
        "TE"
      ]
    },
    {
      "name": "SUPERFLEX",
      "types": [
        "QB",
        "RB",
        "WR",
        "TE"
      ]
    },
    {
      "name": "K",
      "types": [
        "K"
      ]
    },
    

In [5]:

# === Data loading ===
def load_players_from_csv(path="data/template_players.csv") -> pd.DataFrame:
    df = pd.read_csv(path)
    if "risk_score" not in df.columns: df["risk_score"] = 0.0
    df["risk_score"] = df["risk_score"].fillna(0.0).clip(0, 1)
    df.columns = [c.strip().lower() for c in df.columns]
    df["pos"] = df["pos"].str.upper()
    for col in ["proj_pts", "auction_value"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    if "yahoo_adp_rank" in df.columns:
        df["yahoo_adp_rank"] = pd.to_numeric(df["yahoo_adp_rank"], errors="coerce")
    return df

# Optional web fetchers (best-effort)
import requests
from bs4 import BeautifulSoup

def _download_csv(url: str, out_path: str):
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        f.write(r.content)
    return out_path

def fetch_fpros_half_ppr_rankings_csv(out_path="data/fpros_half_ppr.csv"):
    url = "https://www.fantasypros.com/nfl/rankings/half-point-ppr-cheatsheets.php?export=csv"
    return _download_csv(url, out_path)

def fetch_fpros_position_projections(scoring="HALF", pos="qb", out_path="data/fpros_proj_qb.csv"):
    url = f"https://www.fantasypros.com/nfl/projections/{pos}.php?scoring={scoring}&week=draft&csv=1"
    return _download_csv(url, out_path)

def fetch_fpros_auction_values_manual():
    print("Open FantasyPros Auction Calculator in a browser (12 teams, Half-PPR, Superflex, $200), export table, save as data/fpros_auction.csv")

def fetch_yahoo_adp_csv_manual():
    print("Export Yahoo ADP (overall, 2025 half-PPR) to CSV as data/yahoo_adp.csv with columns: player, pos, yahoo_adp_rank")

def assemble_master_table(
    rankings_csv: Optional[str] = None,
    projections_by_pos: Optional[Dict[str, str]] = None,
    auction_csv: Optional[str] = None,
    yahoo_adp_csv: Optional[str] = None
) -> pd.DataFrame:
    if projections_by_pos:
        frames = []
        for pos, path in projections_by_pos.items():
            if path and os.path.exists(path):
                dfp = pd.read_csv(path)
                dfp.columns = [c.strip().lower() for c in dfp.columns]
                cand = [c for c in dfp.columns if c in ("fantasypts", "fpts", "points", "fantasy points")]
                if cand:
                    dfp["proj_pts"] = pd.to_numeric(dfp[cand[0]], errors="coerce")
                elif "projected points" in dfp.columns:
                    dfp["proj_pts"] = pd.to_numeric(dfp["projected points"], errors="coerce")
                else:
                    for seasonal in ["fantasypts", "fpts", "points_total", "season_pts"]:
                        if seasonal in dfp.columns:
                            dfp["proj_pts"] = pd.to_numeric(dfp[seasonal], errors="coerce") / 17.0
                            break
                rename = {"player": "player", "name": "player", "team": "team", "pos": "pos", "position": "pos"}
                for k,v in rename.items():
                    if k in dfp.columns:
                        dfp[v] = dfp[k]
                dfp["pos"] = dfp.get("pos", pos).astype(str).str.upper()
                frames.append(dfp[["player","pos","proj_pts"]].dropna())
        base = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=["player","pos","proj_pts"])
    elif rankings_csv and os.path.exists(rankings_csv):
        base = pd.read_csv(rankings_csv)
        base.columns = [c.strip().lower() for c in base.columns]
        for c in ["proj pts", "proj_pts", "points", "fpts"]:
            if c in base.columns:
                base["proj_pts"] = pd.to_numeric(base[c], errors="coerce")
                break
        if "player" not in base.columns and "player name" in base.columns:
            base["player"] = base["player name"]
        if "pos" not in base.columns and "position" in base.columns:
            base["pos"] = base["position"]
        base["pos"] = base["pos"].astype(str).str.upper()
        base = base[["player","pos","proj_pts"]].dropna()
    else:
        base = load_players_from_csv("data/template_players.csv")[["player","pos","proj_pts"]]

    if auction_csv and os.path.exists(auction_csv):
        auc = pd.read_csv(auction_csv)
        auc.columns = [c.strip().lower() for c in auc.columns]
        if "auction_value" not in auc.columns:
            for c in ["value", "auction $", "$", "price"]:
                if c in auc.columns:
                    auc["auction_value"] = auc[c]
                    break
        if "player" not in auc.columns:
            if "name" in auc.columns:
                auc["player"] = auc["name"]
        auc["pos"] = auc.get("pos", "").astype(str).str.upper()
        base = base.merge(auc[["player","pos","auction_value"]], on=["player","pos"], how="left")

    if yahoo_adp_csv and os.path.exists(yahoo_adp_csv):
        adp = pd.read_csv(yahoo_adp_csv)
        adp.columns = [c.strip().lower() for c in adp.columns]
        if "yahoo_adp_rank" not in adp.columns:
            for c in ["adp", "rank", "overall"]:
                if c in adp.columns:
                    adp["yahoo_adp_rank"] = adp[c]
                    break
        if "player" not in adp.columns:
            for c in ["name","player name"]:
                if c in adp.columns:
                    adp["player"] = adp[c]
        adp["pos"] = adp.get("pos","").astype(str).str.upper()
        base = base.merge(adp[["player","pos","yahoo_adp_rank"]], on=["player","pos"], how="left")

    base["proj_pts"] = pd.to_numeric(base["proj_pts"], errors="coerce").fillna(0.0)
    if "auction_value" not in base.columns: base["auction_value"] = np.nan
    base["auction_value"] = pd.to_numeric(base["auction_value"], errors="coerce")
    if "yahoo_adp_rank" not in base.columns: base["yahoo_adp_rank"] = np.nan
    base["risk_score"] = base.get("risk_score", 0.0)
    base["risk_score"] = pd.to_numeric(base["risk_score"], errors="coerce").fillna(0.0).clip(0,1)

    keep_pos = set(["QB","RB","WR","TE","K","DST"])
    base = base[base["pos"].isin(keep_pos)].reset_index(drop=True)
    return base


In [6]:

# === Market builder ===
def apply_inflation(df: pd.DataFrame, config: dict) -> pd.DataFrame:
    df = df.copy()
    gv = config.get("global_inflation", 1.0)
    pos_infl = config.get("pos_inflation", {})
    def infl(row):
        return gv * pos_infl.get(row["pos"], 1.0)
    df["price_base"] = df["auction_value"]
    df["price_inflated"] = df["auction_value"] * df.apply(infl, axis=1)
    return df

def fit_adp_implied_curve(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    has_adp = d["yahoo_adp_rank"].notna().sum() > 0
    if has_adp:
        d["rank_for_curve"] = d["yahoo_adp_rank"]
    else:
        d["rank_for_curve"] = d["price_inflated"].rank(ascending=False, method="first")
    d = d.sort_values("rank_for_curve")
    d["implied_price_from_adp"] = d["price_inflated"].interpolate(method="linear").bfill().ffill()
    return d

def prob_get_at_price(bid: float, implied_price: float, width: float=0.15, aggression: float=0.0) -> float:
    if np.isnan(implied_price) or implied_price <= 0:
        return 1.0 if bid >= 1 else 0.5
    center = implied_price * (1.0 + aggression)
    x = (bid - center) / max(1e-6, width * implied_price)
    return 1.0 / (1.0 + math.exp(-x))

def build_market(df: pd.DataFrame, config: dict) -> pd.DataFrame:
    d = apply_inflation(df, config)
    d = fit_adp_implied_curve(d)
    rw = config.get("risk_weight", 0.0)
    d["eff_pts"] = d["proj_pts"] * (1.0 - rw * d["risk_score"])
    d["bid_cap"] = d["price_inflated"]
    if config.get("use_adp_probability", True):
        d["p_get_cap"] = d.apply(lambda r: prob_get_at_price(r["bid_cap"], r["implied_price_from_adp"],
                                                             config.get("adp_sigmoid_width", 0.15),
                                                             config.get("adp_aggression", 0.0)), axis=1)
    else:
        d["p_get_cap"] = 1.0
    return d


In [7]:

# === Optimizer ===
import pulp

def optimize_roster(market: pd.DataFrame, config: dict, random_seed: int = 42):
    np.random.seed(random_seed)
    m = pulp.LpProblem("auction_superflex_optimizer", pulp.LpMaximize)

    players = market.reset_index(drop=True).copy()
    players["idx"] = players.index

    x = pulp.LpVariable.dicts("roster", players["idx"].tolist(), lowBound=0, upBound=1, cat=pulp.LpBinary)

    slots = config["starting_slots"]
    slot_names = [s["name"] for s in slots]
    y = {(i, s): pulp.LpVariable(f"start_{i}_{s}", lowBound=0, upBound=1, cat=pulp.LpBinary)
         for i in players["idx"] for s in slot_names}

    bench_w = config.get("bench_weight", 0.25)
    p_get = players["p_get_cap"].fillna(1.0).values
    eff_pts = players["eff_pts"].values

    starter_points = pulp.lpSum([ y[(i,s)] * eff_pts[i] * p_get[i] for i in players["idx"] for s in slot_names ])
    bench_points = pulp.lpSum([ (x[i] - pulp.lpSum([y[(i,s)] for s in slot_names])) * eff_pts[i] * p_get[i] for i in players["idx"] ])
    m += starter_points + bench_w * bench_points

    prices = players["bid_cap"].fillna(players["price_inflated"]).fillna(1.0).values
    m += pulp.lpSum([ x[i] * prices[i] for i in players["idx"] ]) <= config["budget"]

    total_roster = len(slots) + config["bench_spots"]
    m += pulp.lpSum([ x[i] for i in players["idx"] ]) == total_roster

    for s in slot_names:
        m += pulp.lpSum([ y[(i,s)] for i in players["idx"] ]) == 1

    for i in players["idx"]:
        m += pulp.lpSum([ y[(i,s)] for s in slot_names ]) <= x[i]

    def eligible(pos: str, types: List[str]) -> bool:
        return (pos in types)

    for i,row in players.iterrows():
        pos = row["pos"]
        for sdef in slots:
            s = sdef["name"]
            types = sdef["types"]
            if not eligible(pos, types):
                m += y[(i,s)] == 0

    qb_idx = players.index[players["pos"]=="QB"].tolist()
    if qb_idx:
        m += pulp.lpSum([ x[i] for i in qb_idx ]) >= config.get("min_qbs", 2)

    for pos, cnt in config.get("lock_positions", {}).items():
        pos_idx = players.index[players["pos"]==pos].tolist()
        if pos_idx:
            m += pulp.lpSum([ x[i] for i in pos_idx ]) == int(cnt)

    m.solve(pulp.PULP_CBC_CMD(msg=False))

    players["rostered"] = [pulp.value(x[i]) for i in players["idx"]]
    start_map = {}
    for s in slot_names:
        start_map[s] = []
        for i in players["idx"]:
            if pulp.value(y[(i,s)]) > 0.5:
                start_map[s].append(i)
    players["start_slot"] = None
    for s, idxs in start_map.items():
        for i in idxs:
            players.loc[i, "start_slot"] = s

    spent = float(sum(players.loc[players["rostered"]>0.5, "bid_cap"]))
    exp_pts_starters = float(sum(players.loc[players["start_slot"].notna(), "eff_pts"] * players.loc[players["start_slot"].notna(),"p_get_cap"]))
    exp_pts_bench = float(sum(players.loc[(players["rostered"]>0.5) & (players["start_slot"].isna()), "eff_pts"] * players.loc[(players["rostered"]>0.5) & (players["start_slot"].isna()),"p_get_cap"]))

    return {
        "players": players,
        "spent": spent,
        "expected_points_starters": exp_pts_starters,
        "expected_points_bench": exp_pts_bench,
        "objective": pulp.value(m.objective)
    }


In [12]:

# === Scenario utilities ===
from datetime import datetime

def run_scenario(data_csv=None,
                 rankings_csv=None,
                 auction_csv=None,
                 yahoo_adp_csv=None,
                 config_overrides: dict = None,
                 note: str = "Base"):
    cfg = json.loads(json.dumps(CONFIG))
    if config_overrides:
        for k,v in config_overrides.items():
            if isinstance(v, dict) and k in cfg:
                cfg[k].update(v)
            else:
                cfg[k] = v

    df = assemble_master_table(
        rankings_csv=rankings_csv,
        projections_by_pos=None,
        auction_csv=auction_csv,
        yahoo_adp_csv=yahoo_adp_csv
    )
    if df.empty:
        df = load_players_from_csv(data_csv)

    market = build_market(df, cfg)
    result = optimize_roster(market, cfg)

    print(f"--- Scenario: {note} ---")
    print(f"Spent: ${result['spent']:.1f} / {cfg['budget']}")
    print(f"Expected weekly starter points: {result['expected_points_starters']:.2f}")
    print(f"Bench (weighted {cfg['bench_weight']:.2f}) contribution: {result['expected_points_bench']:.2f}")
    rostered = result["players"][result["players"]["rostered"]>0.5].copy()
    rostered = rostered.sort_values(["start_slot","pos","proj_pts"], ascending=[True, True, False])
    display_cols = ["player","pos","proj_pts","risk_score","price_inflated","bid_cap","p_get_cap","start_slot"]
    try:
        from caas_jupyter_tools import display_dataframe_to_user
        display_dataframe_to_user("Optimized Roster — " + note, rostered[display_cols])
    except Exception:
        print(rostered[display_cols].to_string(index=False))

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = f"outputs/roster_{note}_{ts}.csv".replace(" ", "_")
    rostered.to_csv(out_path, index=False)
    print(f"Saved roster to {out_path}")
    return result


In [13]:

# === Simple charts ===
def plot_value_per_point(market: pd.DataFrame):
    d = market.copy()
    d = d[(~d["price_inflated"].isna()) & (d["proj_pts"]>0)]
    d["vpp"] = d["price_inflated"] / d["proj_pts"]
    plt.figure()
    for pos in ["QB","RB","WR","TE"]:
        dd = d[d["pos"]==pos]
        if not dd.empty:
            plt.scatter(dd["proj_pts"], dd["vpp"], label=pos, alpha=0.6)
    plt.xlabel("Projected points per game")
    plt.ylabel("$ per point (inflated price / proj pts)")
    plt.title("Value per Point by Position")
    plt.legend()
    plt.show()

def plot_position_spend(result: dict):
    d = result["players"].copy()
    d = d[d["rostered"]>0.5]
    spend_by_pos = d.groupby("pos")["bid_cap"].sum().sort_values(ascending=False)
    plt.figure()
    spend_by_pos.plot(kind="bar")
    plt.ylabel("Dollars")
    plt.title("Spend by Position (Optimal Roster)")
    plt.show()


In [15]:

# === Preset scenarios (uncomment to run after you load real data) ===

# Base
base_result = run_scenario(rankings_csv="data/fp_rankings_08142025.csv", auction_csv="data/fpros_auction.csv", yahoo_adp_csv="data/yahoo_adp.csv", note="Base")

# Elite-QB tilt
elite_qb = run_scenario(auction_csv="data/fpros_auction.csv", yahoo_adp_csv="data/yahoo_adp.csv",
    config_overrides={"pos_inflation": {"QB": 1.25}, "min_qbs": 3, "bench_weight": 0.15},
    note="Elite_QB")

# Balanced
# balanced = run_scenario(auction_csv="data/fpros_auction.csv", yahoo_adp_csv="data/yahoo_adp.csv",
#     config_overrides={"pos_inflation": {"QB": 1.10}},
#     note="Balanced")

# WR-heavy
# wr_heavy = run_scenario(auction_csv="data/fpros_auction.csv", yahoo_adp_csv="data/yahoo_adp.csv",
#     config_overrides={"pos_inflation": {"WR": 1.10, "RB": 0.95}},
#     note="WR_Heavy")

# Zero-RB tilt
# zero_rb = run_scenario(auction_csv="data/fpros_auction.csv", yahoo_adp_csv="data/yahoo_adp.csv",
#     config_overrides={"pos_inflation": {"WR": 1.15, "TE": 1.05, "RB": 0.90}},
#     note="Zero_RB")

# Risk-averse
# risk_averse = run_scenario(auction_csv="data/fpros_auction.csv", yahoo_adp_csv="data/yahoo_adp.csv",
#     config_overrides={"risk_weight": 0.25},
#     note="Risk_Averse")


--- Scenario: Base ---
Spent: $nan / 200
Expected weekly starter points: 77.24
Bench (weighted 0.25) contribution: 68.73
            player pos  proj_pts  risk_score  price_inflated  bid_cap  p_get_cap start_slot
    Jonathan Mingo  WR      0.49         0.0             NaN      NaN        0.5        DST
    Bijan Robinson  RB     17.16         0.0             NaN      NaN        0.5       FLEX
    Desmond Ridder  QB      0.00         0.0             NaN      NaN        0.5          K
     Lamar Jackson  QB     22.09         0.0             NaN      NaN        0.5         QB
      Jahmyr Gibbs  RB     16.93         0.0             NaN      NaN        0.5        RB1
    Saquon Barkley  RB     17.30         0.0             NaN      NaN        0.5        RB2
        Josh Allen  QB     22.16         0.0             NaN      NaN        0.5  SUPERFLEX
      Brock Bowers  TE     11.92         0.0             NaN      NaN        0.5         TE
  Justin Jefferson  WR     15.22         0.0       

ValueError: Invalid file path or buffer object type: <class 'NoneType'>